# メモ)Python使ったことない人向け
- print("文字列") でデバッグ用の標準出力ができます。"\n"で改行できます。
- インデントは半角スペース4つがお作法。
    - 行の先頭に余計な空白などがあるとダメです。
    - タブキーに半角スペース4つを割り当てても良い。
- import文を使って各種ライブラリを参照します。
    - anacondaインストールならpandasはimport文で呼ぶだけです。
    - pandasはpdと略すのが世の慣習です。(NumPyはnp)
- コメントアウト1行は「#」を、複数行は「＂」か「’」を3つ続けて打ちます。
- 変数に型宣言はありません。

***
# メモ)Jupyter Notebook
- ツールバーの"View">"Toggle Line Number"を選択すると、コードの行番号が出ます
- "Cell">"Run All"　で全セルのプログラムが実行されます。(遅れて来られた方など一気に実行したい人はこちらをどうぞ)
- アウトプット行が煩わしくなってきたら、"Cell">"All Output" > "Clear"などで消すこともできます。
***
# 本教材の進め方
- コードの各セル(グレーの部分)を選択して、上の実行ボタン(▲と|)をポチポチします
    - あえてめんどくさくしているのは、居眠り防止のためです。
    - と言いたいところですが、単純にコンテンツが長くなりすぎるためです。

***
## 補足
-　import pandas as pd　やCSV読込は、1つのプログラムの先頭で1回呼ぶだけでOKです。
- ただし今回は途中参加の方や、実行中にPCが落ちて再起動した場合など焦ることもあるので、あえてJupyterの1セル単位でimportをしています。

***
## 懺悔
- 資料中のクォーテーション表記や、DataFrameのカラム表記に一貫していない箇所があります。。


***
# 第1章 初めの一歩
## 1) ゼロからpandasのデータを作る

### 1-1) Series(一元配列型のデータ)を作る

In [ ]:
import pandas as pd

#値のみのSeries
obj = pd.Series([110.8,  1]) 
print(obj)

#左に出てくる0,1がインデックス

### 1-2) DataFrame(二元配列型のデータ)を作る

In [ ]:
import pandas as pd

#DataFrameはdfと略すことが多いです
df = pd.DataFrame({ '月' : pd.Timestamp('20190501'), '通貨' : ['USD', 'JPY'],'金額' : [110.8,1] })

print(df)
# カラムはデフォルトで、アルファベットの昇順で出てきます。任意の順番に並び替えることもできます。

## 2) CSVからDataFrameをつくる


In [ ]:
import pandas as pd

csv_file="./iris.csv"
df=pd.read_csv(csv_file) #デフォルトのエンコードはutf-8。　encoding="SHIFT-JIS"とすればSHIFT-JISも読める。

print(df[:5]) #先頭5レコードだけ出力

***
## 3) サンプルを動かす

### 3-1) SQL例文との比較
    SELECT 
      Name, avg(SepalLength),avg(PetalLength)

    FROM (iris.csvのデータ)

    WHERE 
      SepalLength >3 and PetalLength < 6

    GROUP BY Name

    ORDER BY Name ASC

In [ ]:
import pandas as pd

#FROM句 csvを読み込む
df=pd.read_csv("./iris.csv") #デフォルトのエンコードはutf-8
print("\n#FROM句----------")
print(df[:10])#先頭10レコードだけ出力


#SELECT句、カラムを絞る および列順の指定 
df=df[["Name","SepalLength","PetalLength"]]
print("\n#SELECT句--------------------")
print(df[:10])#先頭10レコードだけ出力


#WHERE句 条件を絞る
df=df[df["SepalLength"]>3]
df=df[df["PetalLength"]<6]
print("\n#WHERE句--------------------")
print(df[:10])

#GROUP BY句 条件で集計する &ORDER BY句 Name順に並び替える
df=df.groupby(["Name"]).mean().sort_index()
print("\n#GROUP BY句&ORDER BY句----------")
print(df)


#ついでにCSV出力
df.to_csv("集計結果.csv")



### 補足) .to_csv() の引数

- path_or_buf => 出力するファイル名。省略した場合は、コンソール上に文字列として出力されます。
- sep => 区切り文字 (デフォルト: , (カンマ) )
- index => 行名を出力するかどうか。Falseを指定した場合、行名は出力されません。(デフォルト: True)
- encoding => 出力する際の文字コード。’utf-8′, ‘shift_jis’, ‘euc_jp’ などを指定。


***
# 第2章 SQL文法と比較
## 1) FROM的なもの

- CSV以外に、JSONやSQLの結果なども利用できる。
- DataFrameの形にさえ変換できれば自由に加工できる。
- JOINなどで複数のデータを参照する場合は、それぞれDataFrameを読み込んでから加工する。  
    *　各DataFrameは、JOINの前に、集計や不要カラムの除外などを行い、できる限り小さくすると良い。
- サイズの大きなデータは、"chunk"という機能を使って、n行単位で小分けにして読み込むと負荷が少ない。
    * メモリが少ない環境などで有用。
    
## 2) LIMIT的なもの
- df[:10]       
    * limit 10と同じ意味。先頭10行を取得する
- df[10:]      
    * 先頭10〜最終行を取得する
- df[10:30]  
    * 先頭10〜30行を取得する

In [ ]:
# 注) Pythonのループなど出てくるので、ここは参考に読む程度で良い。
'''
chunkを用いた例 
'''

import pandas as pd

#5行ずつデータを読み込む
df_chunks = pd.read_csv("./iris.csv", chunksize=5, dtype="object")

#5行ずつ読み込んだDataFrameから、所定のカラムのみ取得してlist型の変数に格納
pieces = [df[["Name","SepalLength","PetalLength"]] for df in df_chunks]

#list型変数に格納された複数のDataFrameを結合(Unionのように、縦に重ねる)
s =  pd.concat(pieces)

print(s[:15]) #先頭15レコード出力(実際は150)

***
## 3) SELECT的なもの

### A. 基本形(1列のみ指定)
- df['Name']
    * メリット) カラム名の空白OK。複数列OK。日本語カラムOK。
    * デメリット) 長い。

- df.Name
    * メリット) 短く書ける 
    * デメリット) カラム名の空白NG。複数列NG。日本語カラムNG。

### B. 複数列指定
- df[['Name', 'SepalLength, 'PetalLength']]   
    * 注意点) 複数選択は[[]] で括る
 
※カラム名の代わりに番号も使える(使いどころはないかも)。    
例) df[0]   , df[[0 ,2 ,4]]    
※distinctは別の文法を使うので後述。

In [ ]:
import pandas as pd

#それぞれ列を指定して、1行目だけを表示
df= pd.read_csv("./iris.csv")

print("\n # Name列その1")
print(df["Name"][:1])

print("\n # Name列その2")
print(df.Name[:1])

print("\n # 複数列の表示")
print(df[["Name", "SepalLength", "PetalLength"]][:1])


### C. 応用形(loc による行・列指定) <-今覚えなくてOK
- 行・列をラベル名で同時指定して抽出できる。( index 名,column名の順) 
※今回は割愛　　　　

- 複数行・複数列の抽出の場合は「:」を使う　　
　　
- 「:」だけの場合は"列指定の省略(行のみ抽出)"または、"行指定の省略(列のみ抽出)"となる

df =df.loc[:,['Name', 'SepalLength','PetalLength']]

In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")
print(df[:5])
print("\n #index=1-2を表示")
print(df.loc[1:2,["Name", "SepalLength","PetalLength"]])



print("\n #全ての行を表示")
print(df.loc[:,["Name", "SepalLength","PetalLength"]])


***
## 4) GROUP BY句的なもの
### A. 基本) 1つのキーに対して、1列以上の集計を行う
- index=’Name’ 、　value = "SepalLength","PetalLength"各々の最大値
    * df[["Name",  "SepalLength","PetalLength"]] .groupby(["Name"]).max() 

### B. 応用) キーを指定せず、全てのレコードに対して集計を行う
- 列全てに対して集計を行う(例:最大値)
    * df.max()

- 特定の列に対して集計を行う(例:最大値)
    * df.max().列名
    * df.列名.max()

### 注目) GROUP BYをすると、集約されたキーがindexになる。

In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")
print("# NameでGroupBYし、MAXを出力")
print("\n必要なカラムをあらかじめ選択した場合")
print(df[["Name",  "SepalLength","PetalLength"]] .groupby(["Name"]).max() )
print("\nName以外全部集計して出す場合")
print(df.groupby(["Name"]).max() )

print("\n # indexの指定なし--------------------")
print(df.max())

print("\n # SepalLengthの最大だけ出力--------------------")
print(df.max().SepalLength)
print(df.SepalLength.max())

### C. 応用)複数のキーでGROUP BY & 複数の要約統計量
df = df.groupby(['key1','key2']).max()

### D. 応用)複数の要約統計量
df.describe()

In [ ]:
# 応用)複数のキーでGROUP BY
import pandas as pd
df= pd.read_csv("./iris.csv")

#任意にカラム追加
df['Group']="A"  #defaultで全行に"A"
df['Group'][5:9]="B" # index5-9に"B"
df['Group'][10:14]="C" # index10-14に"C"

print("\n # max()のみ出力------------")
print(df.groupby(['Group','Name']).max())
print("\n #複数の要約統計量を出力------------")
print( df.groupby(['Group','Name']).describe())

#ちょっと警告出るけどスルーしてください

***
## 5) ORDER BY的なもの

- valueでソートする    
    * df.sort_values()
- indexでソートする
    * df.sort_index()　
 
    
### A. sort_values(値の順 / 行方向のみ)
- 1つのカラムで昇順ソート
     * df.sort_values(by='key1') 
- 複数カラムで降順ソート
    * df.sort_values(by=[’key1’,’key2’],ascending=False ) ) 

### B. sort_index(index順 / 行方向 )
- indexの昇順ソート
     * df.sort_index(axis=0,ascending=True) 
- indexの降順ソート
    * df.sort_index(axis=0,ascending=False) 

※axis=1でカラムの順序の並び替えになる


In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")

print("\n #'PetalLength','PetalWidth'の降順TOP5")
df=df.sort_values(by=['PetalLength','PetalWidth'],ascending=False ) 
print(df[:5])


print("\n # GroupByの平均を出力------------")
df=df.groupby(['Name']).mean()
print(df)
print("\n # GroupByの平均を出力=> indexの降順------------")
print(df.sort_index(axis=0,ascending=False))


***
## 6) WHERE的なもの(文字列編)  
### 6-1) 完全一致
#### A. イコール 
df[ df.列名=='hoge' ]

#### B. Not イコール
df[ df.列名!='hoge' ]  または  df[~(df.列名=='hoge' )]


In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")

print("\n #Nameが'Iris-virginica'")
print(df[df.Name=='Iris-virginica' ][:5])

print("\n #Nameが'Iris-virginica'以外")
print(df[~(df.Name=='Iris-virginica' )][:5])


### 6-2) 複数条件
#### A. in 
df[ df.列名.isin(['hoge', 'piyo']) ]  
#### B. Not in
df[~(df.列名.isin(['hoge', 'piyo']) )]


※ “~”が否定を示す

In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")

print("\n #Nameが'Iris-virginica'または'Iris-versicolor' ")
print(df[df.Name.isin(['Iris-virginica','Iris-versicolor']) ][:5])

print("\n #Nameが'Iris-virginica'または'Iris-versicolor' ")
print(df[~(df.Name.isin(['Iris-virginica','Iris-versicolor']) )][:5])


### 6-3) 部分一致(〜を含む)
#### A. like 
df[df.列名.str.contains('hoge')]


#### B. Not like
df[~(df.列名.str.contains('hoge'))]

### 6-4) 部分一致の複数条件("|"を間に挟む)
df[df.列名.str.contains('hoge|piyo')]  
    
※str.contains()はPythonの文法。正規表現も扱える。



In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")

print("\n #Nameにvirを含む ")
print(df[df.Name.str.contains('vir') ])

print("\n #Nameにvirを含まない ")
print(df[~(df.Name.str.contains('vir') )])


print("\n #Nameにset|またはcolorを含む ")
print(df[df.Name.str.contains('set|color') ])

### 6-5) 複合条件
#### 条件A and B
df[(df.column_a.str.contains('hoge')) & ~(df.column_b=='piyo' )]    
 => column_aに'hoge'を含み、かつ、column_bに'piyo'を含まない。
 
#### 条件 A or B
df[(df.column_a.str.contains('hoge')) | ~(df.column_b=='piyo' )]    
 => column_aに'hoge'を含む、または、type_bに'piyo'を含まない。

※それぞれの条件は”()”で括る必要がある。   
※”&”と”|”は、1つが正解。


In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")

#条件A and B
print("\n # Nameに'ver'を含み、かつ、SepalLength>6.5。")
df_1=df[(df.Name.str.contains('ver')) & (df.SepalLength>6.5) ] 
print(df_1)

#条件 A or B
print("\n # Nameに’ver'を含まない、または、SepalLength>6.5。")
df_2=df[~(df.Name.str.contains('ver')) | (df.SepalLength>6.5)] 
print(df_2)

***
## 7) WHERE的なもの(数字編)  

### 7-1)イコール
df[df.列名==1000]

### 7-2)以上
df[df.列名>=1000]

### 7-3)以下
df[df.列名<=1000]

#### 他、より大きい、より小さい などが使える。


In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")

print("\n #SepalWidth==4")
print(df[df.SepalWidth==4])

print("\n #SepalWidth>4")
print(df[df.SepalWidth>4])

print("\n #SepalWidth<=2")
print(df[df.SepalWidth<=2])


## 重要) 抽出カラムに数字以外のデータ(NaN)があると、意図しない挙動になることがある
#### 数字列にNullが含まれると、、
カラムは数値型のままで、Nullが"NaN"で表示される。   
それによって、集計がおかしくなったり、後々の処理でエラーになったりするなどバグの温床になりやすい。  
=> 苦しむ前に変換しましょう。(そんなに苦労はない)

#### 数字列に文字列が含まれると、、
カラム自体がobject型になる    
=> これは変換も厄介なので割愛。そもそもの取り込みデータを見直した方が良い。

In [ ]:
# Nullを含むCSVを使った場合
import pandas as pd
df=pd.read_csv("./iris_nan_error.csv" ,sep=',')
print(df[:3])

print("\n # NaN含んだ状態で平均値を出力-------")
print(df.groupby(['Name']).mean())

print("\n # NaNを0埋めして、平均値を出力-------")
df=df.fillna(0)
print(df.groupby(['Name']).mean())


## そもそもNaNとは何か?
"not a number”である
- PythonのNone、欠損値が該当する。
- csvやDBのデータなど、Nullが含まれているとNaNになる。

### 対処法
1. NumPy.nan を判定に使って、該当データをあらかじめreplaceする。
    - 例) df=df.replace(np.nan, 0)
2. df.fillna(0)で、欠損値を穴埋めする
3. df.dropna() で、NaNを含む行を削除する。
    - SQLのwhere 列名 is not null と同義

※必ずしも0に置き換えて良いとは限らない。データを確認し、検討の上で変換すること。

### 応用
特定のカラムを指定したreplaceなども可能。    
文字列のカラムの欠損には"-"を入れるなど使い分けができる。  
df[['SepalLength ', 'SepalWidth']].replace( np.nan,0)

In [ ]:
import numpy as np
import pandas as pd
df=pd.read_csv("./iris_nan_error.csv" ,sep=',')

print("\n # NaNを0に置換-------")
print(df.replace(np.nan, 0)[:3]) 

print("\n # NaNを0で埋める-------")
print(df.fillna(0)[:3])

print("\n # NaNの行を除外-------")
print(df.dropna()[:3]) 


### 実はdf.query()というものも存在する
わかりやすいが速度が遅いとかなんとか。。    
興味があれば検索してみてください。

***
# JOINの説明の前に、ちょっと小ワザ集
## 小ワザ1) 要素の追加 
存在しない列に対して代入をすると、列が追加される    
`df[‘abc] = df.列名a + "_" + df.列名b`

In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")

df["new_column"]=df.Name + "_hoge" 
print(df[:1])

***
## 小ワザ2) 型の変換  (astype)
数値型を文字列型に変換    
df[列名].astype(‘str’)　

In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")

print("\n # 元のデータ型-------")
print(df.SepalLength.dtype)

print("\n # 文字列に変換した後のデータ型-------")
print(df.SepalLength.astype("str").dtype)

***
## 小ワザ3) カラムのリネーム

### 1列のみ  
df.rename(columns ={"列A": "a"})

### 複数
df.rename(columns ={"列A": "a", "列B": "b"})

In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")
print("\n #列名変換前")
print(df[:3])

df=df.rename(columns={"SepalLength":"a", "SepalWidth":"b" })
print("\n #列名変換後")
print(df[:3])

***
## 小ワザ4) 重複データの削除
### 全ての列が重複したものを排除する
df.drop_duplicates()


### 特定のカラムに絞って利用するとdistinctになる。
df.drop_duplicates(["列A", "列B"])


In [ ]:
import pandas as pd
df= pd.read_csv("./iris_nan_error.csv")

#任意にカラム追加
df['Group']="A"  #defaultで全行に"A"
df['Group'][5:9]="B" # index5-9に"B"
df['Group'][10:14]="C" # index10-14に"C"

# NaNを0に置換
print("#重複除外前------- ")
df=df.replace(np.nan, 0)
print("\n  レコード数:　　　"+ str(df.index))
print(df[:10])

print("#完全重複を削除------- ")
df_1= df.drop_duplicates()
print("\n  レコード数:　　　"+ str(df_1.index))
print(df_1[:10])

print("#NameとGroupのdistinct------- ")
df_2= df[["Name", "Group"]].drop_duplicates(["Name", "Group"])
print(df_2)

***
## 小ワザ5) dataframeのインデックスに任意の名前をつける

df.index.name= ‘id’


In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")
print("\n #元のデータ")
print(df[:3])

print("\n #indexに名前をつける")
df.index.name= "id"
print(df[:3])


***
## 小ワザ6) dataFrameのカラムを、indexとして使用する。

df.index = df.pop("列名")    
※前述の列追加で任意に作成した列などに有効

In [ ]:
"""
ちょっと無理矢理感のあるデータになってしまったのですが、、
DBのデータなどでユニークIDとNo列があって、「結合キーにはNoを使いたいんだ!!」という場合を想定してください。
"""

import pandas as pd
df= pd.read_csv("./iris.csv")

#任意にカラム追加
df['Group']="A"  #defaultで全行に"A"
df['Group'][5:9]="B" # index5-9に"B"
df['Group'][10:14]="C" # index10-14に"C"

print("\n #元のデータ+Group")
print(df[:3])

print("\n #仮のカラム作成")
df["new_column"]=df.index.astype(str)+"_"+df["Group"]
print(df[:3])

df.index = df.pop("new_column")
print(df[:3])

*** 
# (再び)第2章 SQL文法と比較
## 8) JOIN的なもの


## 8-1) join
- インデックスでの結合のみ可能。

## 8-2) merge
- インデックス以外に、データをキーにした結合が可能。(これだけでもOK)

## 8-3) concat
- インデックスでの結合のみ可能。
-　縦にも横にも結合可能。unionするならこれを使う。


### 以下、Mergeに絞って説明します

***
## merge の使い方
### A. インデックスをキーにする場合
pd.merge(df1, df2 , left_index=True ,right_index=True )
- onではなく、left_index・right_indexを使用する。
- idでgroup_byした2つのdataframeを結合する時などに使う。

In [ ]:
import pandas as pd
df= pd.read_csv("./iris.csv")

print("\n #index + カラム2つ のdataframe")
df_1=df[["SepalLength", "SepalWidth","Name"]]
print(df_1[:5])

print("\n #index + 別のカラム2つ のdataframe")
df_2=df[["PetalLength", "PetalWidth","Name"]]
print(df_2[:5])

print("\n #index でマージした結果")
df_3= pd.merge(df_1, df_2 , left_index=True ,right_index=True )
print(df_3[:5])

***
### B. カラムをキーにする場合
pd.merge(df1, df2 , on='key' , how='inner')
- ここでの'key'は2つのdataframeに共通するデータの列名を示している。
- howには 'inner' , 'outer' , 'left' , 'right' を指定。
- 指定がない場合は'inner'になる。


In [ ]:
import pandas as pd

print("\n #カラム'No' を持つDataframe")
df_1= pd.read_csv("./iris.csv")
df_1= df_1[["SepalLength", "SepalWidth","Name"]]#列を絞る
df_1["No"]=df_1.index.astype(str)
print(df_1[:3])

print("\n #カラム'ID' を持つDataframe")
df_2= pd.read_csv("./iris.csv")
df_2= df_2[["PetalLength", "PetalWidth","Name"]]#列を絞る
df_2["No"]=df_2.index.astype(str)
print(df_2[:3])


print("\n #カラム'No'と'ID' でマージした結果")
df_3= pd.merge(df_1, df_2, how='inner')
print(df_3[:3])


***
### 応用)キーの列名が、2つのdataframeで異なる場合
pd.merge(df1, df2, left_on=’id’, right_on=’no’)
- 2つのキーの型が異なる場合は、merge前に型の変換が必要
- 例:  df[no].astype(‘str’) 


In [ ]:
import pandas as pd

print("\n #カラム'No' を持つDataframe")
df_1= pd.read_csv("./iris.csv")
df_1["No"]=df_1.index.astype(str)
print(df_1[:3])

print("\n #カラム'ID' を持つDataframe")
df_2= pd.read_csv("./iris.csv")
df_2["ID"]=df_2.index.astype(str)
print(df_2[:3])

print("\n #カラム'No'と'ID' でマージした結果")
df_3= pd.merge(df_1, df_2, left_on='No', right_on='ID')
print(df_3[:3])

#同じ名前のカラムには"_x","_y"がつきます